## Equal-Weight S&P 500 Index Fund

In [2]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.2 MB/s eta 0:00:00


In [3]:
import numpy as np #The Numpy numerical computing library #it is faster than python because library is built of c or c++
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for
import math #The Python math module

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [5]:
IEX_CLOUD_API_TOKEN = 'sk_cf3780ff59204ed2aaeb77f8fcbd1866'

In [10]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(api_url)
data = response.json()
print(data)


{'avgTotalVolume': 64598739, 'calculationPrice': 'close', 'change': 0.76, 'changePercent': 0.0045, 'close': 169.58, 'closeSource': 'official', 'closeTime': 1712347200333, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 169.655, 'delayedPriceTime': 1712347194914, 'extendedChange': -0.065, 'extendedChangePercent': -0.00038, 'extendedPrice': 169.515, 'extendedPriceTime': 1712361599166, 'high': 170.39, 'highSource': '15 minute delayed price', 'highTime': 1712347200000, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 169.68, 'iexCloseTime': 1712347197127, 'iexLastUpdated': 1712347197127, 'iexMarketPercent': 0.014482069110082535, 'iexOpen': 169.51, 'iexOpenTime': 1712323801169, 'iexRealtimePrice': 169.68, 'iexRealtimeSize': 100, 'iexVolume': 609765, 'lastTradeTime': 1712347199767, 'latestPrice': 169.58, 'latestSource': 'Close', 'latestTime': 'April 5, 2024', 'latestUpdate': 1712347200333, 'latestVolume': 42104826, 'low': 168.82, 'lowSource': '

In [13]:
print(data['latestPrice'])
print(data['marketCap'])

169.58
2618634179980


In [15]:
my_columns = ['Ticker', 'Stock Price', ' Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [26]:
# Create a DataFrame from the data
new_data = pd.DataFrame([[symbol, data['latestPrice'], data['marketCap'], 'N/A']], columns=my_columns)

# Concatenate new_data with final_dataframe
final_dataframe = pd.concat([final_dataframe, new_data], ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,169.58,2618634179980,N/A
1,AAPL,169.58,2618634179980,N/A


In [59]:
# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [60]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        # Check if the symbol exists in the data
        if symbol in data:
            # Check if quote information is available for the symbol
            if 'quote' in data[symbol]:
                latest_price = data[symbol]['quote']['latestPrice']
                market_cap = data[symbol]['quote']['marketCap']
                # Create a DataFrame with the data for the current symbol
                new_data = pd.DataFrame([[symbol, latest_price, market_cap, 'N/A']], columns=my_columns)
                # Concatenate new_data with final_dataframe
                final_dataframe = pd.concat([final_dataframe, new_data], ignore_index=True)
            else:
                print(f"No quote information found for symbol: {symbol}")
        else:
            print(f"Symbol '{symbol}' not found in data")

# Display the final DataFrame
final_dataframe

Symbol 'ABC' not found in data
Symbol 'ANTM' not found in data
Symbol 'DISCA' not found in data
Symbol 'FBHS' not found in data
Symbol 'HFC' not found in data
Symbol 'NLOK' not found in data
Symbol 'RE' not found in data
Symbol 'VIAC' not found in data
Symbol 'WLTW' not found in data


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,144.12,42235127530,N/A
1,AAL,13.76,9009453788,N/A
2,AAP,77.34,4610196487,N/A
3,AAPL,169.58,2618634179980,N/A
4,ABBV,170.00,301009987110,N/A
...,...,...,...,...
491,YUM,136.10,38289867708,N/A
492,ZBH,127.33,26161406556,N/A
493,ZBRA,290.38,14924815052,N/A
494,ZION,42.08,6210966762,N/A


## Calculating the Number of Shares to Buy

In [67]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:100000


In [68]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,Number Of Shares to Buy
0,A,144.12,42235127530,N/A,1.0
1,AAL,13.76,9009453788,N/A,14.0
2,AAP,77.34,4610196487,N/A,2.0
3,AAPL,169.58,2618634179980,N/A,1.0
4,ABBV,170.00,301009987110,N/A,1.0
...,...,...,...,...,...
491,YUM,136.10,38289867708,N/A,1.0
492,ZBH,127.33,26161406556,N/A,1.0
493,ZBRA,290.38,14924815052,N/A,0.0
494,ZION,42.08,6210966762,N/A,4.0


## Formatting Our Excel Output

In [82]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [83]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [84]:
# Access the workbook object from XlsxWriter
workbook  = writer.book
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

print(writer.sheets['Recommended Trades'])

In [85]:
for column in column_formats.keys():
    print(f"Column: {column}, Header: {column_formats[column][0]}, Format: {column_formats[column][1]}")

Column: A, Header: Ticker, Format: <xlsxwriter.format.Format object at 0x7f9eae771c90>
Column: B, Header: Price, Format: <xlsxwriter.format.Format object at 0x7f9eae772c80>
Column: C, Header: Market Capitalization, Format: <xlsxwriter.format.Format object at 0x7f9eae772c80>
Column: D, Header: Number of Shares to Buy, Format: <xlsxwriter.format.Format object at 0x7f9eae7711e0>


In [86]:
# Close the workbook to save the file
workbook.close()

In [87]:
from google.colab import files

files.download('recommended_trades.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>